In [9]:
#PES2UG23CS647
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

In [10]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

Tricky Math Problem

In [11]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


In [12]:
#PES2UG23CS647
# 2. CoT Prompt (Magic Phrase)
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the new tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


In [13]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

Tree of Thoughts (ToT)

In [14]:
#PES2UG23CS647
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a child psychologist, I would recommend **Solution 3: Create a "Veggie Face" on Their Plate** as the most sustainable approach to encouraging a 5-year-old to eat vegetables. Here's why:

1. **Promotes creativity and imagination**: This approach allows children to engage in creative play, which is essential for their cognitive, social, and emotional development. By encouraging them to use their imagination, you're fostering a sense of ownership and agency over their mealtime experience.
2. **Non-food rewards**: Unlike the "Vegetable Treasure Hunt" game, which involves hiding vegetables in foods they already enjoy, this approach doesn't rely on the use of food as a reward. This means that you're not inadvertently creating an expectation that they'll only eat vegetables if they're hidden in other foods.
3. **Focus on the process, not just the outcome**: By creating a "Veggie Face," you're focusing on the process of preparing and interacting with fo

Graph of Thoughts (GoT)

In [15]:
#PES2UG23CS647
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
In "Echoes of Eternity," a reclusive physicist, Emma Taylor, discovers a groundbreaking technology that allows her to traverse the fabric of time. However, her first experiment goes horribly wrong, and she finds herself reliving the same fateful night over and over, trapped in a never-ending cycle of time loops. As she desperately tries to break the curse, she begins to notice subtle changes in her interactions with her ex-fiancé, Max, who she had thought she'd lost forever. But with each iteration, Emma starts to question her own sanity, and the lines between reality and fantasy blur. As the loops intensify, a dark presence begins to stir in the shadows, feeding on Emma's growing desperation and threatening to erase not just her existence, but also the love she shares with Max. Can Emma find a way to escape the time loop, reclaim her destiny, and defeat the malevolent force that's hunting her, or will she be forever trapped in a labyrinth of echo

## 4. Summary & Comparison Table

| Method          | Structure                              | Best For...                          | Cost/Latency |
|-----------------|----------------------------------------|--------------------------------------|---------------|
| Simple Prompt   | Input -> Output                        | Simple facts, summaries              | ⭐ Low        |
| CoT (Chain)     | Input -> Steps -> Output               | Math, Logic, Debugging               | ⭐⭐ Medium     |
| ToT (Tree)      | Input -> 3x Branches -> Select -> Output | Strategic decisions, Brainstorming   | ⭐⭐⭐ High      |
| GoT (Graph)     | Input -> Branch -> Mix/Aggregate -> Output | Creative Writing, Research Synthesis | ⭐⭐⭐⭐ Very High |
